**Fixed Effects Model**

In [5]:
%pip install statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 50.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 21.2 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install pandas matplotlib seaborn statsmodels openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 9.1 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
#Dependencies
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns


In [14]:
# Load your Excel sheet
df = pd.read_excel("LLM_Risk_Data.xlsx")

# Check basic structure
print(df.head())
print(df.info())
print(df.groupby(['Patient_ID', 'Condition']).size())

  Patient_ID Condition  AddedVars  Trial   Risk_Score
0        P00  Baseline          0      1          5.5
1        P00  Baseline          0      2          5.0
2        P00  Baseline          0      3          5.5
3        P00  Baseline          0      4          5.0
4        P00  Baseline          0      5          5.5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Patient_ID   180 non-null    object 
 1   Condition    180 non-null    object 
 2   AddedVars    180 non-null    int64  
 3   Trial        180 non-null    int64  
 4    Risk_Score  180 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 7.2+ KB
None
Patient_ID  Condition
P00         Baseline     5
            add_1        5
            add_6        5
P01         Baseline     5
            add_1        5
            add_6        5
P02         Baseline     5


In [16]:
import statsmodels.formula.api as smf

# Treat NumVars as numeric for a trend test (0, 1, 6)
model = smf.mixedlm("Risk_Score ~ AddedVars", df, groups=df["Patient_ID"])
result = model.fit()
print(result.summary())


PatsyError: Error evaluating factor: NameError: name 'Risk_Score' is not defined
    Risk_Score ~ AddedVars
    ^^^^^^^^^^